Week 6-TASK

File Ingestion and Schema Validation:
Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

Read the file ( Present approach of reading the file )

Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

Perform basic validation on data columns : eg: remove special character , white spaces from the col name

As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of read and write file, column name in YAML

Validate number of columns and column name of ingested file with YAML.

Write the file in pipe separated text file (|) in gz format.

Create a summary of the file:

Total number of rows,

total number of columns

file size

In [1]:
import warnings
warnings.filterwarnings('ignore') 
import os

In [2]:
# Determine the size of the raw file:
os.path.getsize(r"D:\DataGlacier\Week6\test.csv")

6927930

In [3]:
import os
file_path1 = r"D:\DataGlacier\Week6\test.csv"

def get_size(file_path1, unit='bytes'):
    file_size = os.path.getsize(file_path1)
    exponents_map = {'bytes': 0, 'kb': 1, 'mb': 2, 'gb': 3}
    if unit not in exponents_map:
        raise ValueError("Must select from \
        ['bytes', 'kb', 'mb', 'gb']")
    else:
        size = file_size / 1024 ** exponents_map[unit]
        return round(size, 3)


print(get_size(file_path1,'mb'),'MB')


6.607 MB


Read the files with various approaches:

In [4]:
import time
import csv
with open(r"D:\DataGlacier\Week6\test.csv") as csvfile:  
    df = csv.DictReader(csvfile)
start_time = time.time()
end_time = time.time()

In [5]:
#Read with DictReader

star_time=time.time()
dfdict = csv.DictReader(open(r"D:\DataGlacier\Week6\test.csv"))
a=("DictReader takes %s seconds" % (time.time() - start_time))
print(a)

DictReader takes 0.00800013542175293 seconds


In [6]:
#Read with Pandas

import pandas as pd
start_time = time.time()
end_time = time.time()
dfpandas = pd.read_csv(open(r"D:\DataGlacier\Week6\test.csv"))
b=("Pandas takes %s seconds" % (time.time() - start_time))
print(b)

Pandas takes 0.23427867889404297 seconds


In [7]:
#Read with Dask

import dask.dataframe
start_time=time.time()
dfdask = dask.dataframe.read_csv(r"D:\DataGlacier\Week6\test.csv")
c=("DaskDataframe takes %s seconds" % (time.time() - start_time))
print(c)

DaskDataframe takes 0.015623092651367188 seconds


In [8]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})
start_time=time.time()
dfmod=pd.read_csv(r"D:\DataGlacier\Week6\test.csv",chunksize=200000)
for chunksize in dfmod:
#print(chunksize)
 d=("Modin and Ray takes %s seconds" % (time.time() - start_time))
print(d)


2023-04-11 16:36:43,362	INFO worker.py:1553 -- Started a local Ray instance.


Modin and Ray takes 0.546745777130127 seconds


In [9]:
print(a)
print(b)
print(c)
print(d)

DictReader takes 0.00800013542175293 seconds
Pandas takes 0.23427867889404297 seconds
DaskDataframe takes 0.015623092651367188 seconds
Modin and Ray takes 0.546745777130127 seconds


In [10]:
print("The least time taken to read a csv file is: ", min(a,b,c,d))

The least time taken to read a csv file is:  DaskDataframe takes 0.015623092651367188 seconds


In [11]:
import dask.dataframe
df= dask.dataframe.read_csv(r"D:\DataGlacier\Week6\test.csv",delimiter=',')


In [12]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 11 entries, id to Vintage
dtypes: object(3), float64(3), int64(5)

In [13]:
df.describe()

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage
npartitions=1,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...


Basic validation:

In [14]:
# Remove special characters
df.columns=df.columns.str.replace('[#,@,&,_]','')


In [15]:
# Remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [16]:
newdfdask=df.columns
newdfdask

Index(['id', 'Gender', 'Age', 'DrivingLicense', 'RegionCode',
       'PreviouslyInsured', 'VehicleAge', 'VehicleDamage', 'AnnualPremium',
       'PolicySalesChannel', 'Vintage'],
      dtype='object')

In [17]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0



Overwriting testutility.py


In [18]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: test
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
     -id
     -Gender
     -Age 
     -DrivingLicense 
     -RegionCode
     -PreviouslyInsured
     -VehicleAge
     -VehicleDamage
     -AnnualPremium
     -PolicySalesChannel 
     -Vintage

Overwriting file.yaml


In [19]:
# Reading config file

import testutility as util
config_data = util.read_config_file('file.yaml')

In [20]:
#data of config file
config_data['inbound_delimiter']

','

In [21]:
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'test',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': '-id -Gender -Age -DrivingLicense -RegionCode -PreviouslyInsured -VehicleAge -VehicleDamage -AnnualPremium -PolicySalesChannel -Vintage'}

In [22]:
# Reading process of the file using Dask
from dask import dataframe as dd
df_sample = dd.read_csv(r"D:\DataGlacier\Week6\test.csv",delimiter=',')
df_sample.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,381110,Male,25,1,11.0,1,< 1 Year,No,35786.0,152.0,53
1,381111,Male,40,1,28.0,0,1-2 Year,Yes,33762.0,7.0,111
2,381112,Male,47,1,28.0,0,1-2 Year,Yes,40050.0,124.0,199
3,381113,Male,24,1,27.0,1,< 1 Year,Yes,37356.0,152.0,187
4,381114,Male,27,1,28.0,1,< 1 Year,No,59097.0,152.0,297


In [23]:
#Reading the file using config file
file_type = config_data['file_type']
#source_file = "D:/DataGlacier/Week6/" + config_data['file_name'] + f'.{file_type}'
source_file="./" + config_data['file_name'] + f'.{file_type}'

In [24]:
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,381110,Male,25,1,11.0,1,< 1 Year,No,35786.0,152.0,53
1,381111,Male,40,1,28.0,0,1-2 Year,Yes,33762.0,7.0,111
2,381112,Male,47,1,28.0,0,1-2 Year,Yes,40050.0,124.0,199
3,381113,Male,24,1,27.0,1,< 1 Year,Yes,37356.0,152.0,187
4,381114,Male,27,1,28.0,1,< 1 Year,No,59097.0,152.0,297


In [25]:
#validating the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['policy_sales_channel', 'vintage', 'vehicle_age', 'id', 'age', 'previously_insured', 'region_code', 'annual_premium', 'gender', 'driving_license', 'vehicle_damage']
Following YAML columns are not in the file uploaded ['c', 'v', 's', 'g', 'n', 'y', 'd', 'e', 'p', ' ', 'r', 'u', 'a', 'h', '-', 'm', 'l', 'i', 't', 'o']


0

In [26]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['id', 'gender', 'age', 'driving_license', 'region_code',
       'previously_insured', 'vehicle_age', 'vehicle_damage', 'annual_premium',
       'policy_sales_channel', 'vintage'],
      dtype='object')
columns of YAML are: -id -Gender -Age -DrivingLicense -RegionCode -PreviouslyInsured -VehicleAge -VehicleDamage -AnnualPremium -PolicySalesChannel -Vintage


In [27]:

if util.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation failed
Following File columns are not in the YAML file ['policy_sales_channel', 'vintage', 'vehicle_age', 'id', 'age', 'previously_insured', 'region_code', 'annual_premium', 'gender', 'driving_license', 'vehicle_damage']
Following YAML columns are not in the file uploaded ['c', 'v', 's', 'g', 'n', 'y', 'd', 'e', 'p', ' ', 'r', 'u', 'a', 'h', '-', 'm', 'l', 'i', 't', 'o']
validation failed


In [28]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv(r'D:\DataGlacier\Week6\test.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('test.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz\\0.part']

In [29]:
#number of files in gz format folder
import os
entries = os.listdir(r'C:/Users/NammaPC/File ingestion and schema validation/test.csv.gz')
for entry in entries:
    print(entry)

0.part


In [30]:
#size of the gz format folder
a=os.path.getsize(r'C:\Users\NammaPC\File ingestion and schema validation/test.csv.gz\\0.part')
a

1541258

In [31]:
import os
file_path = r'C:\Users\NammaPC\File ingestion and schema validation/test.csv.gz\\0.part'

def get_size(file_path, unit='bytes'):
    file_size = os.path.getsize(file_path)
    exponents_map = {'bytes': 0, 'kb': 1, 'mb': 2, 'gb': 3}
    if unit not in exponents_map:
        raise ValueError("Must select from \
        ['bytes', 'kb', 'mb', 'gb']")
    else:
        size = file_size / 1024 ** exponents_map[unit]
        return round(size, 3)


sizeoffile=(get_size(file_path,'mb'))


In [32]:
print("Summary:")

# Total number of observations:
print("Total number of rows are:",len(df))

# Total number of features:
print("Total number of columns are :",len(df.columns))
df.columns

print("Size of the file is",sizeoffile,"MB")


Summary:
Total number of rows are: 127037
Total number of columns are : 11
Size of the file is 1.47 MB
